# Differential Evolution for Feature Selection

This notebook implements Differential Evolution (DE) for feature selection on benchmark metaheuristic datasets.

## Overview
- Continuous representation internally, converted to binary feature subsets
- Differential mutation (DE/rand/1/bin strategy)
- Binomial crossover
- Greedy selection
- Comprehensive visualizations of the evolutionary process


## 1. Imports and Setup


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef
from sklearn.datasets import (load_breast_cancer, load_wine, load_iris, make_classification,
                             load_digits, load_diabetes, fetch_covtype, load_linnerud)
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")


## 2. Dataset Loading and Preprocessing


In [ ]:
def load_benchmark_dataset(dataset_name='breast_cancer'):
    """
    Load benchmark datasets for feature selection
    
    Args:
        dataset_name: Name of the dataset
    
    Returns:
        X: Feature matrix
        y: Target vector
        feature_names: List of feature names
    """
    if dataset_name == 'breast_cancer':
        data = load_breast_cancer()
        X, y = data.data, data.target
        feature_names = data.feature_names
    elif dataset_name == 'wine':
        data = load_wine()
        X, y = data.data, data.target
        feature_names = data.feature_names
    elif dataset_name == 'iris':
        data = load_iris()
        X, y = data.data, data.target
        feature_names = data.feature_names
    elif dataset_name == 'digits':
        data = load_digits()
        X, y = data.data, data.target
        feature_names = [f'Pixel_{i+1}' for i in range(X.shape[1])]
    elif dataset_name == 'diabetes':
        data = load_diabetes()
        X, y = data.data, data.target
        # Convert regression to binary classification (threshold at median)
        y = (y > np.median(y)).astype(int)
        feature_names = data.feature_names
    elif dataset_name == 'synthetic_1':
        # Synthetic dataset 1: Small, balanced
        X, y = make_classification(
            n_samples=500,
            n_features=20,
            n_informative=8,
            n_redundant=5,
            n_repeated=0,
            n_classes=2,
            n_clusters_per_class=1,
            random_state=42
        )
        feature_names = [f'Feature_{i+1}' for i in range(X.shape[1])]
    elif dataset_name == 'synthetic_2':
        # Synthetic dataset 2: Medium, multi-class
        X, y = make_classification(
            n_samples=1000,
            n_features=30,
            n_informative=10,
            n_redundant=10,
            n_repeated=0,
            n_classes=3,
            n_clusters_per_class=1,
            random_state=43
        )
        feature_names = [f'Feature_{i+1}' for i in range(X.shape[1])]
    elif dataset_name == 'synthetic_3':
        # Synthetic dataset 3: Large, high-dimensional
        X, y = make_classification(
            n_samples=800,
            n_features=50,
            n_informative=15,
            n_redundant=20,
            n_repeated=0,
            n_classes=2,
            n_clusters_per_class=2,
            random_state=44
        )
        feature_names = [f'Feature_{i+1}' for i in range(X.shape[1])]
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print(f"Dataset: {dataset_name}")
    print(f"  Samples: {X.shape[0]}, Features: {X.shape[1]}, Classes: {len(np.unique(y))}")
    
    return X_scaled, y, feature_names

# Define all benchmark datasets to test
BENCHMARK_DATASETS = [
    'breast_cancer',
    'wine',
    'iris',
    'digits',
    'diabetes',
    'synthetic_1',
    'synthetic_2',
    'synthetic_3'
]

print(f"Available benchmark datasets: {BENCHMARK_DATASETS}")
print(f"Total datasets: {len(BENCHMARK_DATASETS)}")


## 3. Differential Evolution Implementation

### 3.1 Representation and Initialization


In [ ]:
def initialize_population(pop_size, n_features, lower_bound=0.0, upper_bound=1.0):
    """
    Initialize population of continuous vectors
    
    Args:
        pop_size: Population size
        n_features: Number of features
        lower_bound: Lower bound for initialization (default 0.0)
        upper_bound: Upper bound for initialization (default 1.0)
    
    Returns:
        population: Array of shape (pop_size, n_features) with continuous values in [lower_bound, upper_bound]
    """
    population = np.random.uniform(lower_bound, upper_bound, size=(pop_size, n_features))
    return population

def continuous_to_binary(continuous_vector, threshold=0.5):
    """
    Convert continuous vector to binary chromosome
    
    Args:
        continuous_vector: Continuous values in [0, 1]
        threshold: Threshold for conversion (default 0.5)
    
    Returns:
        binary_vector: Binary array
    """
    binary = (continuous_vector >= threshold).astype(int)
    
    # Ensure at least one feature is selected
    if np.sum(binary) == 0:
        # Select the feature with highest continuous value
        binary[np.argmax(continuous_vector)] = 1
    
    return binary

print("Population initialization functions defined!")


### 3.2 Fitness Function


In [ ]:
def evaluate_fitness(continuous_vector, X, y, classifier='rf', cv_folds=5, metric='accuracy', 
                     alpha=1.0, beta=0.0, threshold=0.5):
    """
    Evaluate fitness of a continuous vector (converted to binary feature subset)
    
    Fitness = α * Performance - β * (|X|/n)
    
    Args:
        continuous_vector: Continuous values in [0, 1]
        X: Feature matrix
        y: Target vector
        classifier: Type of classifier ('rf', 'svm', 'knn')
        cv_folds: Number of cross-validation folds
        metric: Performance metric ('accuracy', 'f1', 'mcc', 'roc_auc')
        alpha: Weight for performance term
        beta: Weight for size penalty term
        threshold: Threshold for converting continuous to binary
    
    Returns:
        fitness: Fitness value
        performance: Performance metric value
    """
    # Convert continuous to binary
    chromosome = continuous_to_binary(continuous_vector, threshold=threshold)
    
    # Get selected features
    selected_features = np.where(chromosome == 1)[0]
    
    # If no features selected, return low fitness
    if len(selected_features) == 0:
        return 0.0, 0.0
    
    # Select features
    X_selected = X[:, selected_features]
    
    # Initialize classifier
    if classifier == 'rf':
        clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    elif classifier == 'svm':
        clf = SVC(kernel='rbf', random_state=42, probability=True)
    elif classifier == 'knn':
        clf = KNeighborsClassifier(n_neighbors=5)
    else:
        raise ValueError(f"Unknown classifier: {classifier}")
    
    # Perform cross-validation
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    if metric == 'accuracy':
        scores = cross_val_score(clf, X_selected, y, cv=cv, scoring='accuracy')
        performance = scores.mean()
    elif metric == 'f1':
        scores = cross_val_score(clf, X_selected, y, cv=cv, scoring='f1_macro')
        performance = scores.mean()
    elif metric == 'mcc':
        # MCC requires custom scoring
        scores = []
        for train_idx, val_idx in cv.split(X_selected, y):
            clf.fit(X_selected[train_idx], y[train_idx])
            y_pred = clf.predict(X_selected[val_idx])
            mcc = matthews_corrcoef(y[val_idx], y_pred)
            scores.append(mcc)
        performance = np.mean(scores)
    elif metric == 'roc_auc':
        if len(np.unique(y)) == 2:
            scores = cross_val_score(clf, X_selected, y, cv=cv, scoring='roc_auc')
            performance = scores.mean()
        else:
            # Multi-class ROC-AUC
            scores = cross_val_score(clf, X_selected, y, cv=cv, scoring='roc_auc_ovo')
            performance = scores.mean()
    else:
        raise ValueError(f"Unknown metric: {metric}")
    
    # Compute fitness with optional size penalty
    n_selected = len(selected_features)
    size_penalty = beta * (n_selected / len(chromosome))
    fitness = alpha * performance - size_penalty
    
    return fitness, performance

print("Fitness evaluation function defined!")


In [ ]:
def differential_mutation(population, F=0.5, strategy='rand/1'):
    """
    Differential mutation operator
    
    Args:
        population: Current population (pop_size, n_features)
        F: Scaling factor (default 0.5)
        strategy: Mutation strategy ('rand/1', 'best/1', 'rand/2', 'best/2')
    
    Returns:
        mutant_vectors: Mutated vectors
    """
    pop_size, n_features = population.shape
    mutant_vectors = np.zeros_like(population)
    
    # Find best individual (for best-based strategies)
    # Note: We'll need fitness values to determine best, so for now use first individual
    # This will be updated in the main algorithm
    
    for i in range(pop_size):
        if strategy == 'rand/1':
            # v_i = x_r1 + F * (x_r2 - x_r3)
            indices = np.random.choice(pop_size, size=3, replace=False)
            r1, r2, r3 = indices[0], indices[1], indices[2]
            mutant_vectors[i] = population[r1] + F * (population[r2] - population[r3])
        elif strategy == 'best/1':
            # v_i = x_best + F * (x_r1 - x_r2)
            # For now, use first individual as best (will be updated in main algorithm)
            best_idx = 0  # Will be passed as parameter in actual implementation
            indices = np.random.choice([j for j in range(pop_size) if j != i], size=2, replace=False)
            r1, r2 = indices[0], indices[1]
            mutant_vectors[i] = population[best_idx] + F * (population[r1] - population[r2])
        elif strategy == 'rand/2':
            # v_i = x_r1 + F * (x_r2 - x_r3) + F * (x_r4 - x_r5)
            indices = np.random.choice(pop_size, size=5, replace=False)
            r1, r2, r3, r4, r5 = indices
            mutant_vectors[i] = population[r1] + F * (population[r2] - population[r3]) + F * (population[r4] - population[r5])
        elif strategy == 'best/2':
            # v_i = x_best + F * (x_r1 - x_r2) + F * (x_r3 - x_r4)
            best_idx = 0  # Will be passed as parameter
            indices = np.random.choice([j for j in range(pop_size) if j != i], size=4, replace=False)
            r1, r2, r3, r4 = indices
            mutant_vectors[i] = population[best_idx] + F * (population[r1] - population[r2]) + F * (population[r3] - population[r4])
        else:
            raise ValueError(f"Unknown strategy: {strategy}")
        
        # Clip to bounds [0, 1]
        mutant_vectors[i] = np.clip(mutant_vectors[i], 0.0, 1.0)
    
    return mutant_vectors


def binomial_crossover(target, mutant, CR=0.7):
    """
    Binomial crossover operator
    
    Args:
        target: Target vector (parent)
        mutant: Mutant vector
        CR: Crossover rate (default 0.7)
    
    Returns:
        trial: Trial vector (offspring)
    """
    n_features = len(target)
    trial = target.copy()
    
    # Random index to ensure at least one component from mutant
    j_rand = np.random.randint(n_features)
    
    # Crossover
    for j in range(n_features):
        if np.random.rand() < CR or j == j_rand:
            trial[j] = mutant[j]
    
    # Clip to bounds
    trial = np.clip(trial, 0.0, 1.0)
    
    return trial


def greedy_selection(target, trial, target_fitness, trial_fitness):
    """
    Greedy selection: select the better individual
    
    Args:
        target: Target vector
        trial: Trial vector
        target_fitness: Fitness of target
        trial_fitness: Fitness of trial
    
    Returns:
        selected: Selected vector
        selected_fitness: Fitness of selected vector
    """
    if trial_fitness >= target_fitness:
        return trial.copy(), trial_fitness
    else:
        return target.copy(), target_fitness

print("Differential Evolution operators defined!")


In [ ]:
def compute_spiral_radius(spiral_type, theta, a=1.0, b=1.0, k=1.0):
    """
    Compute radial distance for different spiral types
    
    Args:
        spiral_type: Type of spiral ('archimedean', 'logarithmic', 'fermat', 'hyperbolic', 'euler')
        theta: Angular displacement
        a: Spiral parameter a (default 1.0)
        b: Spiral parameter b (default 1.0, used for Archimedean)
        k: Spiral parameter k (default 1.0, used for Logarithmic)
    
    Returns:
        r: Radial distance
    """
    if spiral_type == 'archimedean':
        # Linear radial growth: r = a + b*theta
        r = a + b * theta
    elif spiral_type == 'logarithmic':
        # Exponential radial growth: r = a * exp(k*theta)
        r = a * np.exp(k * theta)
    elif spiral_type == 'fermat':
        # Square-root expansion: r = a * sqrt(theta)
        r = a * np.sqrt(np.maximum(theta, 0))
    elif spiral_type == 'hyperbolic':
        # Inverse decay: r = a / theta
        r = a / np.maximum(theta, 1e-10)  # Avoid division by zero
    elif spiral_type == 'euler':
        # Euler spiral uses Fresnel integrals, approximate with logarithmic
        # For simplicity, we use a logarithmic approximation
        r = a * np.exp(k * theta)
    else:
        raise ValueError(f"Unknown spiral type: {spiral_type}")
    
    return r

def spiral_update(continuous_vector, elite_vector, spiral_type='archimedean', 
                  r_factor=0.5, theta_factor=2*np.pi, a=1.0, b=1.0, k=1.0):
    """
    Apply spiral-based update to a continuous vector
    
    Generic spiral update: z = x* + R(r,θ)(x - x*)
    where x* is the elite solution and x is the current vector
    
    Args:
        continuous_vector: Current continuous vector (n_features,)
        elite_vector: Elite (best) continuous vector (n_features,)
        spiral_type: Type of spiral ('archimedean', 'logarithmic', 'fermat', 'hyperbolic', 'euler')
        r_factor: Radial scaling factor (default 0.5)
        theta_factor: Angular displacement factor (default 2π)
        a, b, k: Spiral parameters
    
    Returns:
        new_vector: Updated continuous vector
    """
    n_features = len(continuous_vector)
    
    # Step 1: Compute spiral parameters
    # Generate random angular displacement for each dimension
    theta = np.random.uniform(0, theta_factor, size=n_features)
    
    # Compute radial scaling for each dimension
    r = compute_spiral_radius(spiral_type, theta, a=a, b=b, k=k)
    r = r_factor * r  # Scale by r_factor
    
    # Step 2: Apply spiral update: z = x* + R(r,θ)(x - x*)
    # R(r,θ) represents rotation and scaling
    # Compute rotation-scaling factor
    R = r * np.cos(theta) + (1 - r) * np.sin(theta)
    R = np.clip(R, 0, 1)  # Ensure R is in [0,1]
    
    # Apply spiral transformation
    z = elite_vector + R * (continuous_vector - elite_vector)
    
    # Step 3: Clamp z to [0,1]
    z = np.clip(z, 0.0, 1.0)
    
    return z

def apply_spiral_enhancement(population, elite_vector, elite_fitness, 
                            fitness_values, spiral_type='archimedean',
                            spiral_rate=0.3, r_factor=0.5, theta_factor=2*np.pi,
                            a=1.0, b=1.0, k=1.0):
    """
    Apply spiral enhancement to a portion of the population
    
    Args:
        population: Current population
        elite_vector: Best continuous vector (elite solution)
        elite_fitness: Fitness of elite solution
        fitness_values: Fitness values for all individuals
        spiral_type: Type of spiral to use
        spiral_rate: Fraction of population to enhance with spiral (default 0.3)
        r_factor: Radial scaling factor
        theta_factor: Angular displacement factor
        a, b, k: Spiral parameters
    
    Returns:
        enhanced_population: Population with spiral-enhanced individuals
    """
    enhanced_population = population.copy()
    pop_size = len(population)
    
    # Select individuals to enhance (excluding the elite itself)
    # We enhance individuals that are not the elite but have reasonable fitness
    n_to_enhance = max(1, int(spiral_rate * pop_size))
    
    # Get indices sorted by fitness (excluding elite)
    elite_idx = np.argmax(fitness_values)
    candidate_indices = [i for i in range(pop_size) if i != elite_idx]
    
    # Select top candidates for enhancement (those with good but not best fitness)
    if len(candidate_indices) > 0:
        candidate_fitness = [fitness_values[i] for i in candidate_indices]
        sorted_candidates = sorted(zip(candidate_fitness, candidate_indices), reverse=True)
        selected_indices = [idx for _, idx in sorted_candidates[:n_to_enhance]]
        
        # Apply spiral enhancement to selected individuals
        for idx in selected_indices:
            enhanced_population[idx] = spiral_update(
                population[idx], 
                elite_vector,
                spiral_type=spiral_type,
                r_factor=r_factor,
                theta_factor=theta_factor,
                a=a, b=b, k=k
            )
    
    return enhanced_population

print("Spiral-based enhancement operators defined!")


### 3.5 Differential Evolution Main Class


In [ ]:
class DifferentialEvolutionFeatureSelection:
    """
    Differential Evolution for Feature Selection with Spiral-Based Enhancement
    """
    
    def __init__(self, X, y, pop_size=50, n_generations=50, F=0.5, CR=0.7,
                 mutation_strategy='rand/1', classifier='rf', metric='accuracy',
                 alpha=1.0, beta=0.0, cv_folds=5, threshold=0.5,
                 use_spiral=False, spiral_type='archimedean', spiral_rate=0.3,
                 r_factor=0.5, theta_factor=2*np.pi, spiral_a=1.0, spiral_b=1.0, spiral_k=1.0):
        """
        Initialize DE for feature selection
        
        Args:
            X: Feature matrix
            y: Target vector
            pop_size: Population size
            n_generations: Number of generations
            F: Scaling factor for mutation (default 0.5)
            CR: Crossover rate (default 0.7)
            mutation_strategy: Mutation strategy ('rand/1', 'best/1', 'rand/2', 'best/2')
            classifier: Classifier type ('rf', 'svm', 'knn')
            metric: Performance metric ('accuracy', 'f1', 'mcc', 'roc_auc')
            alpha: Weight for performance term
            beta: Weight for size penalty term
            cv_folds: Number of CV folds
            threshold: Threshold for converting continuous to binary (default 0.5)
            use_spiral: Whether to use spiral-based enhancement (default False)
            spiral_type: Type of spiral ('archimedean', 'logarithmic', 'fermat', 'hyperbolic', 'euler')
            spiral_rate: Fraction of population to enhance with spiral (default 0.3)
            r_factor: Radial scaling factor for spiral (default 0.5)
            theta_factor: Angular displacement factor for spiral (default 2π)
            spiral_a: Spiral parameter a (default 1.0)
            spiral_b: Spiral parameter b (default 1.0, used for Archimedean)
            spiral_k: Spiral parameter k (default 1.0, used for Logarithmic)
        """
        self.X = X
        self.y = y
        self.n_features = X.shape[1]
        self.pop_size = pop_size
        self.n_generations = n_generations
        self.F = F
        self.CR = CR
        self.mutation_strategy = mutation_strategy
        self.classifier = classifier
        self.metric = metric
        self.alpha = alpha
        self.beta = beta
        self.cv_folds = cv_folds
        self.threshold = threshold
        
        # Spiral enhancement parameters
        self.use_spiral = use_spiral
        self.spiral_type = spiral_type
        self.spiral_rate = spiral_rate
        self.r_factor = r_factor
        self.theta_factor = theta_factor
        self.spiral_a = spiral_a
        self.spiral_b = spiral_b
        self.spiral_k = spiral_k
        
        # History tracking
        self.history = {
            'best_fitness': [],
            'avg_fitness': [],
            'worst_fitness': [],
            'best_performance': [],
            'avg_performance': [],
            'best_n_features': [],
            'avg_n_features': [],
            'diversity': [],
            'best_vector': None
        }
    
    def compute_diversity(self, population):
        """
        Compute population diversity as average Euclidean distance
        """
        pop_size = len(population)
        if pop_size < 2:
            return 0.0
        
        total_distance = 0
        n_pairs = 0
        
        for i in range(pop_size):
            for j in range(i + 1, pop_size):
                # Euclidean distance
                distance = np.linalg.norm(population[i] - population[j])
                total_distance += distance
                n_pairs += 1
        
        return total_distance / n_pairs if n_pairs > 0 else 0.0
    
    def differential_mutation_with_best(self, population, best_idx):
        """
        Differential mutation with best individual support
        """
        pop_size, n_features = population.shape
        mutant_vectors = np.zeros_like(population)
        
        for i in range(pop_size):
            if self.mutation_strategy == 'rand/1':
                # v_i = x_r1 + F * (x_r2 - x_r3)
                indices = np.random.choice(pop_size, size=3, replace=False)
                r1, r2, r3 = indices[0], indices[1], indices[2]
                mutant_vectors[i] = population[r1] + self.F * (population[r2] - population[r3])
            elif self.mutation_strategy == 'best/1':
                # v_i = x_best + F * (x_r1 - x_r2)
                indices = np.random.choice([j for j in range(pop_size) if j != i], size=2, replace=False)
                r1, r2 = indices[0], indices[1]
                mutant_vectors[i] = population[best_idx] + self.F * (population[r1] - population[r2])
            elif self.mutation_strategy == 'rand/2':
                # v_i = x_r1 + F * (x_r2 - x_r3) + F * (x_r4 - x_r5)
                indices = np.random.choice(pop_size, size=5, replace=False)
                r1, r2, r3, r4, r5 = indices
                mutant_vectors[i] = population[r1] + self.F * (population[r2] - population[r3]) + self.F * (population[r4] - population[r5])
            elif self.mutation_strategy == 'best/2':
                # v_i = x_best + F * (x_r1 - x_r2) + F * (x_r3 - x_r4)
                indices = np.random.choice([j for j in range(pop_size) if j != i], size=4, replace=False)
                r1, r2, r3, r4 = indices
                mutant_vectors[i] = population[best_idx] + self.F * (population[r1] - population[r2]) + self.F * (population[r3] - population[r4])
            else:
                raise ValueError(f"Unknown strategy: {self.mutation_strategy}")
            
            # Clip to bounds [0, 1]
            mutant_vectors[i] = np.clip(mutant_vectors[i], 0.0, 1.0)
        
        return mutant_vectors
    
    def run(self):
        """
        Run the differential evolution algorithm
        """
        print(f"Starting Differential Evolution for Feature Selection")
        print(f"  Population size: {self.pop_size}")
        print(f"  Generations: {self.n_generations}")
        print(f"  Scaling factor (F): {self.F}")
        print(f"  Crossover rate (CR): {self.CR}")
        print(f"  Mutation strategy: {self.mutation_strategy}")
        print(f"  Classifier: {self.classifier}")
        print(f"  Metric: {self.metric}")
        if self.use_spiral:
            print(f"  Spiral Enhancement: ENABLED ({self.spiral_type})")
            print(f"    Spiral rate: {self.spiral_rate}, r_factor: {self.r_factor}")
        else:
            print(f"  Spiral Enhancement: DISABLED")
        print(f"\n{'='*60}")
        
        # Initialize population
        population = initialize_population(self.pop_size, self.n_features)
        
        # Evaluate initial population
        fitness_values = np.zeros(self.pop_size)
        performance_values = np.zeros(self.pop_size)
        
        print("Evaluating initial population...")
        for i in range(self.pop_size):
            fitness_values[i], performance_values[i] = evaluate_fitness(
                population[i], self.X, self.y, 
                classifier=self.classifier, 
                metric=self.metric,
                alpha=self.alpha, 
                beta=self.beta,
                cv_folds=self.cv_folds,
                threshold=self.threshold
            )
            if (i + 1) % 10 == 0:
                print(f"  Evaluated {i+1}/{self.pop_size} individuals")
        
        # Find best solution
        best_idx = np.argmax(fitness_values)
        best_vector = population[best_idx].copy()
        best_fitness = fitness_values[best_idx]
        best_binary = continuous_to_binary(best_vector, threshold=self.threshold)
        
        # Record initial statistics
        self.history['best_fitness'].append(best_fitness)
        self.history['avg_fitness'].append(np.mean(fitness_values))
        self.history['worst_fitness'].append(np.min(fitness_values))
        self.history['best_performance'].append(performance_values[best_idx])
        self.history['avg_performance'].append(np.mean(performance_values))
        self.history['best_n_features'].append(np.sum(best_binary))
        self.history['avg_n_features'].append(np.mean([np.sum(continuous_to_binary(p, threshold=self.threshold)) for p in population]))
        self.history['diversity'].append(self.compute_diversity(population))
        
        print(f"\nGeneration 0: Best fitness = {best_fitness:.6f}, "
              f"Performance = {performance_values[best_idx]:.6f}, "
              f"Features = {np.sum(best_binary)}")
        
        # Main evolution loop
        for gen in range(1, self.n_generations + 1):
            # Create new population
            new_population = population.copy()
            new_fitness = fitness_values.copy()
            new_performance = performance_values.copy()
            
            # For each individual in population
            for i in range(self.pop_size):
                # Mutation
                mutant = self.differential_mutation_with_best(population, best_idx)[i]
                
                # Crossover
                trial = binomial_crossover(population[i], mutant, CR=self.CR)
                
                # Evaluate trial
                trial_fitness, trial_performance = evaluate_fitness(
                    trial, self.X, self.y,
                    classifier=self.classifier,
                    metric=self.metric,
                    alpha=self.alpha,
                    beta=self.beta,
                    cv_folds=self.cv_folds,
                    threshold=self.threshold
                )
                
                # Selection
                if trial_fitness >= fitness_values[i]:
                    new_population[i] = trial
                    new_fitness[i] = trial_fitness
                    new_performance[i] = trial_performance
                    
                    # Update best if improved
                    if trial_fitness > best_fitness:
                        best_vector = trial.copy()
                        best_fitness = trial_fitness
                        best_idx = i
            
            # Update population
            population = new_population
            fitness_values = new_fitness
            performance_values = new_performance
            
            # Apply spiral-based enhancement if enabled
            if self.use_spiral:
                # Store population before spiral enhancement for comparison
                population_before_spiral = population.copy()
                
                # Apply spiral enhancement to a portion of the population
                population = apply_spiral_enhancement(
                    population,
                    best_vector,  # Elite vector
                    best_fitness,  # Elite fitness
                    fitness_values,
                    spiral_type=self.spiral_type,
                    spiral_rate=self.spiral_rate,
                    r_factor=self.r_factor,
                    theta_factor=self.theta_factor,
                    a=self.spiral_a,
                    b=self.spiral_b,
                    k=self.spiral_k
                )
                
                # Re-evaluate enhanced individuals (only those that changed)
                for i in range(self.pop_size):
                    if not np.allclose(population[i], population_before_spiral[i], atol=1e-10):
                        fitness_values[i], performance_values[i] = evaluate_fitness(
                            population[i], self.X, self.y,
                            classifier=self.classifier,
                            metric=self.metric,
                            alpha=self.alpha,
                            beta=self.beta,
                            cv_folds=self.cv_folds,
                            threshold=self.threshold
                        )
                
                # Update best solution after spiral enhancement
                current_best_idx = np.argmax(fitness_values)
                if fitness_values[current_best_idx] > best_fitness:
                    best_vector = population[current_best_idx].copy()
                    best_fitness = fitness_values[current_best_idx]
                    best_idx = current_best_idx
            
            # Convert best to binary for statistics
            best_binary = continuous_to_binary(best_vector, threshold=self.threshold)
            
            # Record statistics
            self.history['best_fitness'].append(best_fitness)
            self.history['avg_fitness'].append(np.mean(fitness_values))
            self.history['worst_fitness'].append(np.min(fitness_values))
            self.history['best_performance'].append(performance_values[best_idx])
            self.history['avg_performance'].append(np.mean(performance_values))
            self.history['best_n_features'].append(np.sum(best_binary))
            self.history['avg_n_features'].append(np.mean([np.sum(continuous_to_binary(p, threshold=self.threshold)) for p in population]))
            self.history['diversity'].append(self.compute_diversity(population))
            
            # Print progress
            if gen % 5 == 0 or gen == self.n_generations:
                print(f"Generation {gen}: Best fitness = {best_fitness:.6f}, "
                      f"Performance = {performance_values[best_idx]:.6f}, "
                      f"Features = {np.sum(best_binary)}, "
                      f"Diversity = {self.history['diversity'][-1]:.2f}")
        
        # Store best solution (convert to binary)
        self.history['best_vector'] = best_vector
        best_chromosome = continuous_to_binary(best_vector, threshold=self.threshold)
        
        print(f"\n{'='*60}")
        print(f"Evolution completed!")
        print(f"Best fitness: {best_fitness:.6f}")
        print(f"Best performance: {performance_values[best_idx]:.6f}")
        print(f"Selected features: {np.sum(best_chromosome)}/{self.n_features}")
        
        return best_chromosome, best_fitness

print("Differential Evolution class defined!")


In [ ]:
# Define all spiral types to test (including 'standard' for comparison)
SPIRAL_TYPES = ['standard', 'archimedean', 'logarithmic', 'fermat', 'hyperbolic', 'euler']

# Spiral enhancement parameters (will be used when spiral_type != 'standard')
spiral_params = {
    'spiral_rate': 0.3,  # Fraction of population to enhance with spiral (30%)
    'r_factor': 0.5,  # Radial scaling factor
    'theta_factor': 2*np.pi,  # Angular displacement factor
    'spiral_a': 1.0,  # Spiral parameter a
    'spiral_b': 1.0,  # Spiral parameter b (used for Archimedean)
    'spiral_k': 1.0  # Spiral parameter k (used for Logarithmic)
}

print("="*80)
print("SPIRAL TYPES CONFIGURATION")
print("="*80)
print(f"Spiral types to test: {SPIRAL_TYPES}")
print(f"  - 'standard': Standard DE without spiral enhancement")
print(f"  - 'archimedean': Linear radial growth (r = a + b*θ)")
print(f"  - 'logarithmic': Exponential radial growth (r = a*e^(k*θ))")
print(f"  - 'fermat': Square-root expansion (r = a*√θ)")
print(f"  - 'hyperbolic': Inverse decay (r = a/θ)")
print(f"  - 'euler': Euler spiral (approximated with logarithmic)")
print(f"\nSpiral parameters: {spiral_params}")
print("="*80)


In [ ]:
# ============================================================================
# SPIRAL ENHANCEMENT CONFIGURATION - AUTOMATIC ALL SPIRAL TYPES
# ============================================================================
# AUTOMATICALLY RUNS ALL SPIRAL TYPES IN A LOOP
# This will test: 'standard', 'archimedean', 'logarithmic', 'fermat', 'hyperbolic', 'euler'

# Define all spiral types to test (including 'standard' for comparison)
SPIRAL_TYPES = ['standard', 'archimedean', 'logarithmic', 'fermat', 'hyperbolic', 'euler']

# Spiral parameters (used for all non-standard spiral types)
spiral_params = {
    'spiral_rate': 0.3,  # Fraction of population to enhance with spiral (30%)
    'r_factor': 0.5,  # Radial scaling factor
    'theta_factor': 2*np.pi,  # Angular displacement factor
    'spiral_a': 1.0,  # Spiral parameter a
    'spiral_b': 1.0,  # Spiral parameter b (used for Archimedean)
    'spiral_k': 1.0  # Spiral parameter k (used for Logarithmic)
}

print("="*80)
print("SPIRAL ENHANCEMENT: AUTOMATIC MODE - ALL SPIRAL TYPES")
print("="*80)
print(f"Will automatically test {len(SPIRAL_TYPES)} spiral types:")
for st in SPIRAL_TYPES:
    print(f"  - {st.upper()}")
print("="*80)

# Configure DE parameters (base parameters, classifier will be set in loop)
de_base_params = {
    'pop_size': 20,
    'n_generations': 10,
    'F': 0.5,  # Scaling factor
    'CR': 0.7,  # Crossover rate
    'mutation_strategy': 'rand/1',  # 'rand/1', 'best/1', 'rand/2', 'best/2'
    'metric': 'accuracy',  # 'accuracy', 'f1', 'mcc', 'roc_auc'
    'alpha': 1.0,  # Performance weight
    'beta': 0.0,  # Size penalty weight (0 = no penalty)
    'cv_folds': 5,
    'threshold': 0.5  # Threshold for continuous to binary conversion
}

# Define all classifiers to test
CLASSIFIERS = ['rf', 'svm', 'knn']

# Storage for all results across datasets, models, and spiral types
# Structure: all_results[dataset_name][classifier_name][spiral_type] = {...}
all_results = {}

baseline_results = {}

print("="*80)
print("STEP 1: BASELINE EVALUATION - ALL CLASSIFIERS ON ALL FEATURES")
print("="*80)
print(f"Datasets: {len(BENCHMARK_DATASETS)}")
print(f"Classifiers: {CLASSIFIERS}")
print(f"Total baseline evaluations: {len(BENCHMARK_DATASETS) * len(CLASSIFIERS)}")
print("="*80)

# Preprocessing: Evaluate all classifiers on all features for all datasets
import time
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for dataset_name in BENCHMARK_DATASETS:
    print(f"\n{'-'*80}")
    print(f"Baseline Evaluation - Dataset: {dataset_name.upper()}")
    print(f"{'-'*80}")
    
    try:
        # Load dataset
        X, y, feature_names = load_benchmark_dataset(dataset_name)
        n_features = X.shape[1]
        
        # Initialize baseline results dictionary for this dataset
        baseline_results[dataset_name] = {}
        
        # Evaluate each classifier on all features
        for classifier_name in CLASSIFIERS:
            try:
                print(f"  Evaluating {classifier_name.upper()} on all features...", end=' ')
                
                # Initialize classifier
                if classifier_name == 'rf':
                    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
                elif classifier_name == 'svm':
                    clf = SVC(kernel='rbf', random_state=42, probability=True)
                elif classifier_name == 'knn':
                    clf = KNeighborsClassifier(n_neighbors=5)
                
                # Evaluate on all features
                baseline_start = time.time()
                scores_all = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
                baseline_time = time.time() - baseline_start
                
                # Store baseline results
                baseline_results[dataset_name][classifier_name] = {
                    'X': X,
                    'y': y,
                    'feature_names': feature_names,
                    'n_features': n_features,
                    'scores_all': scores_all,
                    'performance_all': scores_all.mean(),
                    'performance_std_all': scores_all.std(),
                    'baseline_time': baseline_time
                }
                
                print(f" Performance: {scores_all.mean():.4f} ± {scores_all.std():.4f} (Time: {baseline_time:.2f}s)")
                
            except Exception as e:
                print(f"✗ Error: {str(e)}")
                import traceback
                traceback.print_exc()
                continue
        
    except Exception as e:
        print(f"\n✗ Error loading dataset {dataset_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*80}")
print("STEP 1 COMPLETED: Baseline evaluation finished")
print(f"{'='*80}")

print("\n" + "="*80)
print("STEP 2: RUNNING DIFFERENTIAL EVOLUTION ON ALL BENCHMARK DATASETS AND MODELS")
print("="*80)
print(f"Datasets: {len(BENCHMARK_DATASETS)}")
print(f"Classifiers: {CLASSIFIERS}")
print(f"Spiral Types: {SPIRAL_TYPES}")
print(f"Total combinations: {len(BENCHMARK_DATASETS) * len(CLASSIFIERS) * len(SPIRAL_TYPES)}")
print("="*80)

# Loop through all spiral types
for spiral_type in SPIRAL_TYPES:
    print(f"\n{'#'*80}")
    print(f"# SPIRAL TYPE: {spiral_type.upper()}")
    print(f"{'#'*80}")
    
    # Determine if we should use spiral enhancement
    use_spiral = (spiral_type != 'standard')
    
    # Loop through all benchmark datasets
    for dataset_name in BENCHMARK_DATASETS:
        print(f"\n{'='*80}")
        print(f"Processing Dataset: {dataset_name.upper()}")
        print(f"{'='*80}")

        try:
            # Load dataset
            X, y, feature_names = load_benchmark_dataset(dataset_name)
            n_features = X.shape[1]

            # Initialize results dictionary for this dataset if not exists
            if dataset_name not in all_results:
                all_results[dataset_name] = {}

            # Loop through all classifiers
            for classifier_name in CLASSIFIERS:
                print(f"\n{'-'*80}")
                print(f"  Classifier: {classifier_name.upper()} | Spiral: {spiral_type.upper()}")
                print(f"{'-'*80}")

                try:
                    # Start timing
                    import time
                    start_time = time.time()

                    # Create DE parameters with current classifier and spiral settings
                    de_params = de_base_params.copy()
                    de_params['classifier'] = classifier_name
                    de_params['use_spiral'] = use_spiral

                    # Add spiral-specific parameters if using spiral
                    if use_spiral:
                        de_params['spiral_type'] = spiral_type
                        de_params['spiral_rate'] = spiral_params['spiral_rate']
                        de_params['r_factor'] = spiral_params['r_factor']
                        de_params['theta_factor'] = spiral_params['theta_factor']
                        de_params['spiral_a'] = spiral_params['spiral_a']
                        de_params['spiral_b'] = spiral_params['spiral_b']
                        de_params['spiral_k'] = spiral_params['spiral_k']

                    # Initialize and run DE
                    de = DifferentialEvolutionFeatureSelection(X, y, **de_params)
                    best_chromosome, best_fitness = de.run()

                    # Get selected features
                    selected_features = np.where(best_chromosome == 1)[0]
                    selected_features_idx = np.where(best_chromosome == 1)[0]
                    X_selected = X[:, selected_features_idx]

                    # End timing
                    de_time = time.time() - start_time

                    # Evaluate performance with selected features using the same classifier
                    if classifier_name == 'rf':
                        clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
                    elif classifier_name == 'svm':
                        clf = SVC(kernel='rbf', random_state=42, probability=True)
                    elif classifier_name == 'knn':
                        clf = KNeighborsClassifier(n_neighbors=5)

                    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

                    # Time the evaluation of selected features
                    eval_start = time.time()
                    scores_selected = cross_val_score(clf, X_selected, y, cv=cv, scoring='accuracy')
                    eval_time = time.time() - eval_start

                    # Get baseline results (all features) from preprocessing step
                    baseline_data = baseline_results[dataset_name][classifier_name]
                    scores_all = baseline_data['scores_all']

                    total_time = de_time + eval_time

                    # Initialize classifier dictionary if not exists
                    if classifier_name not in all_results[dataset_name]:
                        all_results[dataset_name][classifier_name] = {}

                    # Store results with spiral_type
                    all_results[dataset_name][classifier_name][spiral_type] = {
                        'X': X,
                        'y': y,
                        'feature_names': feature_names,
                        'n_features': n_features,
                        'de': de,
                        'best_chromosome': best_chromosome,
                        'best_fitness': best_fitness,
                        'selected_features': selected_features,
                        'selected_features_idx': selected_features_idx,
                        'scores_selected': scores_selected,
                        'scores_all': scores_all,  # From baseline evaluation
                        'performance_selected': scores_selected.mean(),
                        'performance_all': scores_all.mean(),  # From baseline evaluation
                        'n_selected': len(selected_features),
                        'reduction_ratio': len(selected_features) / n_features,
                        'de_time': de_time,
                        'eval_time': eval_time,
                        'total_time': total_time,
                        'baseline_time': baseline_data['baseline_time']  # Store baseline evaluation time
                    }

                    print(f"\n  Completed {dataset_name} with {classifier_name}")
                    print(f"    Selected: {len(selected_features)}/{n_features} features ({len(selected_features)/n_features*100:.1f}%)")
                    print(f"    Performance (Selected): {scores_selected.mean():.4f} ± {scores_selected.std():.4f}")
                    print(f"    Performance (All): {scores_all.mean():.4f} ± {scores_all.std():.4f}")
                    print(f"    DE Time: {de_time:.2f}s, Eval Time: {eval_time:.2f}s, Total: {total_time:.2f}s")

                except Exception as e:
                    print(f"\n  ✗ Error processing {dataset_name} with {classifier_name}: {str(e)}")
                    import traceback
                    traceback.print_exc()
                    continue

            print(f"\n✓ Completed all classifiers for {dataset_name}")

        except Exception as e:
            print(f"\n✗ Error loading dataset {dataset_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue

# Summary
total_combinations = sum(len(all_results[ds]) for ds in all_results.keys())
print(f"\n{'='*80}")
print(f"COMPLETED PROCESSING")
print(f"  Successful combinations: {total_combinations}/{len(BENCHMARK_DATASETS) * len(CLASSIFIERS)}")
print(f"  Datasets processed: {len(all_results)}/{len(BENCHMARK_DATASETS)}")
print(f"{'='*80}")


## 5. Comprehensive Visualizations


In [ ]:
def compare_with_all_features(X, y, best_chromosome, classifier='rf', cv_folds=5):
    """
    Compare performance with selected features vs all features
    """
    # Evaluate with selected features
    selected_features = np.where(best_chromosome == 1)[0]
    X_selected = X[:, selected_features]
    
    # Initialize classifier
    if classifier == 'rf':
        clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    elif classifier == 'svm':
        clf = SVC(kernel='rbf', random_state=42, probability=True)
    elif classifier == 'knn':
        clf = KNeighborsClassifier(n_neighbors=5)
    
    # Cross-validation with selected features
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    scores_selected = cross_val_score(clf, X_selected, y, cv=cv, scoring='accuracy')
    
    # Cross-validation with all features
    scores_all = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Create comparison plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Box plot comparison
    data_to_plot = [scores_all, scores_selected]
    bp = axes[0].boxplot(data_to_plot, labels=['All Features', f'Selected ({len(selected_features)})'], 
                         patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    bp['boxes'][1].set_facecolor('lightgreen')
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].set_title('Performance Comparison: All Features vs Selected Features', 
                      fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='y')

    means = [scores_all.mean(), scores_selected.mean()]
    stds = [scores_all.std(), scores_selected.std()]

    bars = axes[1].bar(
    ['All Features', f'Selected ({len(selected_features)})'],
    means, yerr=stds, capsize=10,
    color=['lightblue', 'lightgreen'],
    edgecolor='black', linewidth=2)

    for bar in bars:
        bar.set_alpha(0.8)

    axes[1].set_ylabel('Mean Accuracy ± Std Dev', fontsize=12)
    axes[1].set_title('Mean Performance Comparison', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')

    low = min(means) - 0.01
    high = max(means) + 0.01
    axes[1].set_ylim([low, high])

    # labels
    for i, (mean, std) in enumerate(zip(means, stds)):
        axes[1].text(i, mean + std + 0.002, f'{mean:.4f}±{std:.4f}',
                     ha='center', fontsize=10, fontweight='bold')

    plt.tight_layout()
    plt.show()

    # Print statistics
    total = X.shape[1]
    selected = np.sum(best_chromosome)
    removed = total - selected
    print(f"\n{'='*60}")
    print("Performance Comparison:")
    print(f"  All Features: {scores_all.mean():.6f} ± {scores_all.std():.6f}")
    print(f"  Selected Features ({len(selected_features)}): {scores_selected.mean():.6f} ± {scores_selected.std():.6f}")
    print(f"  Feature Reduction: {removed}/{X.shape[1]} = { removed /X.shape[1]*100:.1f}%")
    print(f"  Performance Change: {(scores_selected.mean() - scores_all.mean())*100:+.2f}%")
    print(f"{'='*60}")
    
    return scores_all, scores_selected


In [ ]:
def print_summary_statistics(de, best_chromosome, feature_names):
    """
    Print comprehensive summary statistics
    """
    print(f"\n{'='*70}")
    print("DIFFERENTIAL EVOLUTION FEATURE SELECTION - SUMMARY STATISTICS")
    print(f"{'='*70}")
    
    print(f"\n Algorithm Configuration:")
    print(f"  Population Size: {de.pop_size}")
    print(f"  Generations: {de.n_generations}")
    print(f"  Scaling Factor (F): {de.F}")
    print(f"  Crossover Rate (CR): {de.CR}")
    print(f"  Mutation Strategy: {de.mutation_strategy}")
    print(f"  Classifier: {de.classifier}")
    print(f"  Metric: {de.metric}")
    print(f"  CV Folds: {de.cv_folds}")
    print(f"  Threshold: {de.threshold}")
    
    print(f"\n Final Results:")
    final_fitness = de.history['best_fitness'][-1]
    final_performance = de.history['best_performance'][-1]
    n_selected = np.sum(best_chromosome)
    
    print(f"  Best Fitness: {final_fitness:.6f}")
    print(f"  Best Performance: {final_performance:.6f}")
    print(f"  Selected Features: {n_selected}/{de.n_features} ({n_selected/de.n_features*100:.1f}%)")
    
    print(f"\n Evolution Statistics:")
    print(f"  Initial Fitness: {de.history['best_fitness'][0]:.6f}")
    print(f"  Final Fitness: {final_fitness:.6f}")
    print(f"  Fitness Improvement: {final_fitness - de.history['best_fitness'][0]:.6f} "
          f"({(final_fitness/de.history['best_fitness'][0]-1)*100:+.2f}%)")
    print(f"  Initial Features: {de.history['best_n_features'][0]}")
    print(f"  Final Features: {n_selected}")
    print(f"  Feature Reduction: {de.history['best_n_features'][0] - n_selected} "
          f"({(1 - n_selected/de.history['best_n_features'][0])*100:+.1f}%)")
    
    print(f"\n Population Diversity:")
    print(f"  Initial Diversity: {de.history['diversity'][0]:.2f}")
    print(f"  Final Diversity: {de.history['diversity'][-1]:.2f}")
    print(f"  Diversity Change: {de.history['diversity'][-1] - de.history['diversity'][0]:.2f}")
    
    print(f"\n Selected Features:")
    selected_indices = np.where(best_chromosome == 1)[0]
    for i, idx in enumerate(selected_indices):
        print(f"  {i+1:2d}. [{idx:2d}] {feature_names[idx]}")
    
    print(f"\n{'='*70}\n")


In [ ]:
def collect_results(all_results, classifiers):
    """
    Collect results into a DataFrame
    """
    rows = []

    for ds_name, clf_dict in all_results.items():
        for clf_name, spiral_dict in clf_dict.items():
            if clf_name not in classifiers:
                continue

            for spiral_type, result in spiral_dict.items():
                rows.append({
                    "dataset": ds_name,
                    "classifier": clf_name,
                    "spiral": spiral_type,
                    "perf_sel": result["performance_selected"],
                    "perf_all": result["performance_all"],
                    "n_total": result["n_features"],
                    "n_sel": result["n_selected"],
                    "ratio": result["reduction_ratio"],
                    "fit": result["best_fitness"],
                    "de_time": result["de_time"],
                    "eval_time": result["eval_time"],
                    "total_time": result["total_time"]
                })

    return pd.DataFrame(rows)


In [ ]:
CLASSIFIER_FULL = {
    "rf": "Random Forest",
    "svm": "Support Vector Machine",
    "knn": "K-Nearest Neighbors"
}

def create_summary_table(all_results):
    """
    Create a comprehensive summary table of all results across datasets and classifiers
    """
    if len(all_results) == 0:
        print("No results to display!")
        return
    
    data = []

    for ds_name, clf_dict in all_results.items():
        for clf_name, spirals in clf_dict.items():
            clf_pretty = CLASSIFIER_FULL.get(clf_name, clf_name)

            for spiral_type, result in spirals.items():
                data.append({
                    'Dataset': ds_name,
                    'Classifier': clf_pretty,
                    'Spiral': spiral_type,
                    'Total Features': result['n_features'],
                    'Selected Features': result['n_selected'],
                    'Reduction Ratio': result['reduction_ratio'],
                    'Performance All': result['performance_all'],
                    'Performance Selected': result['performance_selected'],
                    'Performance Diff': result['performance_selected'] - result['performance_all'],
                    'Best Fitness': result['best_fitness'],
                    'DE Time (s)': result['de_time'],
                    'Eval Time (s)': result['eval_time'],
                    'Total Time (s)': result['total_time'],
                })

    df = pd.DataFrame(data)
    
    print("\n" + "="*100)
    print("COMPREHENSIVE SUMMARY TABLE - ALL DATASETS")
    print("="*100)
    print(df.to_string(index=False))
    print("="*100)
    
    return df


In [ ]:
def statistical_analysis(all_results):
    """
    Perform statistical analysis across all datasets and classifiers.
    """
    if len(all_results) == 0:
        print("No results to analyze!")
        return

    print("\n" + "="*80)
    print("STATISTICAL ANALYSIS ACROSS ALL DATASETS AND CLASSIFIERS")
    print("="*80)

    dataset_names = []
    classifier_names = []
    spiral_names = []
    performances_selected = []
    performances_all = []
    reduction_ratios = []
    n_features_selected = []
    best_fitnesses = []
    de_times = []
    total_times = []

    for ds_name, clf_dict in all_results.items():
        for clf_name, spiral_dict in clf_dict.items():
            for spiral_type, result in spiral_dict.items():
                dataset_names.append(ds_name)
                classifier_names.append(clf_name)
                spiral_names.append(spiral_type)
                performances_selected.append(result['performance_selected'])
                performances_all.append(result['performance_all'])
                reduction_ratios.append(result['reduction_ratio'])
                n_features_selected.append(result['n_selected'])
                best_fitnesses.append(result['best_fitness'])
                de_times.append(result['de_time'])
                total_times.append(result['total_time'])

    performances_selected = np.array(performances_selected)
    performances_all = np.array(performances_all)
    reduction_ratios = np.array(reduction_ratios)
    n_features_selected = np.array(n_features_selected)
    best_fitnesses = np.array(best_fitnesses)
    de_times = np.array(de_times)
    total_times = np.array(total_times)

    # --- BASIC METRICS ---
    print("\nPerformance Statistics:")
    print(f"  Mean (All Features):       {performances_all.mean():.4f} ± {performances_all.std():.4f}")
    print(f"  Mean (Selected Features):  {performances_selected.mean():.4f} ± {performances_selected.std():.4f}")
    diff = performances_selected - performances_all
    print(f"  Mean Improvement:          {diff.mean():+.4f}")

    print("\nFeature Reduction Statistics:")
    print(f"  Mean Reduction Ratio:      {reduction_ratios.mean():.4f} ± {reduction_ratios.std():.4f}")
    print(f"  Mean Selected Features:    {n_features_selected.mean():.2f} ± {n_features_selected.std():.2f}")
    print(f"  Min Selected Features:     {n_features_selected.min()}")
    print(f"  Max Selected Features:     {n_features_selected.max()}")

    best_idx = np.argmax(best_fitnesses)
    worst_idx = np.argmin(best_fitnesses)

    print("\nFitness Statistics:")
    print(f"  Mean Best Fitness:         {best_fitnesses.mean():.4f} ± {best_fitnesses.std():.4f}")
    print(f"  Best Overall Fitness:      {best_fitnesses[best_idx]:.4f} "
          f"(Dataset: {dataset_names[best_idx]}, Classifier: {classifier_names[best_idx]})")
    print(f"  Worst Overall Fitness:     {best_fitnesses[worst_idx]:.4f} "
          f"(Dataset: {dataset_names[worst_idx]}, Classifier: {classifier_names[worst_idx]})")

    improved = np.sum(diff > 0)
    degraded = np.sum(diff < 0)
    unchanged = np.sum(diff == 0)
    total_cases = len(diff)

    print("\nPerformance Change Analysis:")
    print(f"  Improved:  {improved}/{total_cases} ({improved/total_cases*100:.1f}%)")
    print(f"  Degraded:  {degraded}/{total_cases} ({degraded/total_cases*100:.1f}%)")
    print(f"  Unchanged: {unchanged}/{total_cases} ({unchanged/total_cases*100:.1f}%)")

    print("\nCorrelation Analysis:")
    print(f"  Reduction Ratio vs Perf:   {np.corrcoef(reduction_ratios, performances_selected)[0,1]:.4f}")
    print(f"  Selected Features vs Perf: {np.corrcoef(n_features_selected, performances_selected)[0,1]:.4f}")
    print(f"  Best Fitness vs Perf:      {np.corrcoef(best_fitnesses, performances_selected)[0,1]:.4f}")
    print(f"  Total Time vs # Features:  {np.corrcoef(total_times, n_features_selected)[0,1]:.4f}")

    print("\nAnalysis by Classifier:")
    unique_clfs = sorted(set(classifier_names))
    for clf in unique_clfs:
        idxs = [i for i,c in enumerate(classifier_names) if c == clf]
        clf_perf = performances_selected[idxs]
        clf_time = total_times[idxs]
        print(f"  {clf.upper()}:")
        print(f"    Mean Performance:        {clf_perf.mean():.4f} ± {clf_perf.std():.4f}")
        print(f"    Mean Total Time:         {clf_time.mean():.2f}s ± {clf_time.std():.2f}s")

    print("\n" + "="*80)
